In [1]:
import logging
import datetime

import numpy as np
import torch
from torch.nn.utils import parameters_to_vector
import torch.optim as optim
from torchinfo import summary

import matplotlib.pyplot as plt

import config
import modules.dataloaders as dataloaders

# import modules.model_mobilenetv2 as cnv_model
# import modules.model_mobilenetv2_small as cnv_model
# import modules.model_mobilenetv2_mini_Resnet as cnv_model
import modules.model_mobilenetv2_mini as cnv_model

import modules.loss as loss
import modules.metrics as metrics
import modules.train_epoch as train_epoch
import modules.val_epoch as val_epoch
import modules.utils as utils

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.21 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
# from brevitas.export import export_onnx_qcdq
# from brevitas.export import export_qonnx

# Reimport

In [3]:
# import importlib
# importlib.reload(config)

# Logger

In [4]:
log_path = config.LOGS_FOLDER

logger = logging.getLogger("GonLogger")
logger.propagate = False
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_path + 'logfile.log')
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

logger.info('BED Classifier.\n' +  
            '\tOne Head.\n' +
            '\tWeighted for Precision.\n' +
            '\tBrevitas Default.\n'+ 
            '\tDataset images divided by 255.\n')

# Hyperparameters Log

In [5]:
''' ============================
    Print Config Values
============================ '''
print('\nDatasets Length')
print(f'\tTrain and Val: {"Full" if config.DS_LEN == None else config.DS_LEN}')
print(f'\nLoad Model: {config.LOAD_MODEL}')
if (config.LOAD_MODEL == True):
    print(f'\tModel: {config.LOAD_MODEL_FILE}')
print(f'Device: {config.DEVICE}')
print('Optimizer:')
print(f'\tLearning Rate: {config.LEARNING_RATE}')
print(f'\tWeight Decay: {config.WEIGHT_DECAY}')
print('Scheduler:')
print(f'\tScheduler factor: {config.FACTOR}')
print(f'\tScheduler patience: {config.PATIENCE}')
print(f'\tScheduler threshold: {config.THRES}')
print(f'\tScheduler min learning rate: {config.MIN_LR}')
print(f'Batch Size: {config.BATCH_SIZE}')
print(f'Num Workers: {config.NUM_WORKERS}')
print(f'Pin Memory: {config.PIN_MEMORY}')
print(f'Epochs: {config.EPOCHS}')
print('\nIMG DIMS:')
print(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
print('\nBrevitas Config:')
print(f'\tFixed Point: {config.FIXED_POINT}')
print(f'\tWeights Bit Width: {config.WEIGHTS_BIT_WIDTH}')
print(f'\tBig Layers Weights Bit Width: {config.BIG_LAYERS_WEIGHTS_BIT_WIDTH}')
print(f'\tBias Bit Width: {config.BIAS_BIT_WIDTH}')
print(f'\tActivations Bit Width: {config.ACTIVATIONS_BIT_WIDTH}')

logger.info('\nDatasets Length')
logger.info(f'\tTrain and Val: {"Full" if config.DS_LEN == None else config.DS_LEN}')
logger.info(f'\nLoad Model: {config.LOAD_MODEL}')
if (config.LOAD_MODEL == True):
    logger.info(f'\tModel: {config.LOAD_MODEL_FILE}')
logger.info(f'\nDevice: {config.DEVICE}')
logger.info('Optimizer:')
logger.info(f'\tLearning Rate: {config.LEARNING_RATE}')
logger.info(f'\tWeight Decay: {config.WEIGHT_DECAY}')
logger.info('Scheduler:')
logger.info(f'\tScheduler factor: {config.FACTOR}')
logger.info(f'\tScheduler patience: {config.PATIENCE}')
logger.info(f'\tScheduler threshold: {config.THRES}')
logger.info(f'\tScheduler min learning rate: {config.MIN_LR}')
logger.info(f'\nBatch Size: {config.BATCH_SIZE}')
logger.info(f'Num Workers: {config.NUM_WORKERS}')
logger.info(f'Pin Memory: {config.PIN_MEMORY}')
logger.info(f'Epochs: {config.EPOCHS}')
logger.info('\nIMG DIMS:')
logger.info(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
logger.info('\nBrevitas Config:')
logger.info(f'\tFixed Point: {config.FIXED_POINT}')
logger.info(f'\tWeights Bit Width: {config.WEIGHTS_BIT_WIDTH}')
logger.info(f'\tBig Layers Weights Bit Width: {config.BIG_LAYERS_WEIGHTS_BIT_WIDTH}')
logger.info(f'\tBias Bit Width: {config.BIAS_BIT_WIDTH}')
logger.info(f'\tActivations Bit Width: {config.ACTIVATIONS_BIT_WIDTH}')


Datasets Length
	Train and Val: Full

Load Model: False
Device: cuda
Optimizer:
	Learning Rate: 0.001
	Weight Decay: 0.001
Scheduler:
	Scheduler factor: 0.8
	Scheduler patience: 2
	Scheduler threshold: 0.001
	Scheduler min learning rate: 1e-06
Batch Size: 64
Num Workers: 8
Pin Memory: True
Epochs: 100

IMG DIMS:
	Width: 224
	Height: 224

Brevitas Config:
	Fixed Point: True
	Weights Bit Width: 4
	Big Layers Weights Bit Width: 4
	Bias Bit Width: 4
	Activations Bit Width: 4


# Dataloaders

In [6]:
train_loader = dataloaders.get_train_loader()
val_loader = dataloaders.get_val_loader()


TRAIN DFIRE dataset
DFire Removed wrong images: 0
DFire empty images: 7833
DFire only smoke images: 4681
DFire only fire images: 944
DFire smoke and fire images: 3763

Train DFire dataset len: 17221

TRAIN FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 5994
FASDD only smoke images: 2541
FASDD only fire images: 105
FASDD smoke and fire images: 3911

Train FASDD UAV dataset len: 12551

VAL FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 3995
FASDD only smoke images: 1693
FASDD only fire images: 70
FASDD smoke and fire images: 2607

Val FASDD UAV dataset len: 8365

TRAIN FASDD CV dataset


FASDD Removed wrong images: 0
FASDD empty images: 19600
FASDD only smoke images: 11708
FASDD only fire images: 6276
FASDD smoke and fire images: 10076

Train FASDD CV dataset len: 47660

Val FASDD CV dataset
FASDD Removed wrong images: 0
FASDD empty images: 13066
FASDD only smoke images: 7804
FASDD only fire images: 4183
FASDD smoke and fire images: 6717

Val FASDD CV dataset len: 31770

Concatenate Train DFire and Train FASDD UAV datasets
Train dataset len: 29772
Concatenate with Val FASDD UAV dataset
Train dataset len: 38137
Concatenate with Train FASDD CV dataset
Train dataset len: 85797
Concatenate with Val FASDD CV dataset
Train dataset len: 117567

TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895

Test dataset len: 4306

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 1997
FASDD only smoke images: 846
FASDD only fire images: 35
FASDD smoke a

### Plot Some Train Pictures

In [7]:
for i, (img, label) in enumerate(train_loader):

    plt.subplots(8,4, figsize=(8, 16))
    
    for idx in range(config.BATCH_SIZE):
        plt.subplot(8, 4, idx+1)
        plt.imshow(img[idx].permute(1, 2, 0))
        title = ""
        if label[idx][0] == 1 and label[idx][1] == 1:
            title += "Smoke and Fire"
        elif label[idx][0] == 1 and label[idx][1] == 0:
            title += "Only Smoke"
        elif label[idx][0] == 0 and label[idx][1] == 1:
            title += "Only Fire"
        else:
            title += "Empty"
        plt.title(title)
        
        if (idx == 31):
            break
    plt.tight_layout()
    plt.savefig(config.RUN_FOLDER + 'train_pictures.png')
    #plt.show()
    plt.close()
    break

### Plot Some Val Pictures

In [8]:
for i, (img, label) in enumerate(val_loader):

    plt.subplots(8,4, figsize=(8, 16))
    
    for idx in range(config.BATCH_SIZE):
        plt.subplot(8, 4, idx+1)
        plt.imshow(img[idx].permute(1, 2, 0))
        title = ""
        if label[idx][0] == 1 and label[idx][1] == 1:
            title += "Smoke and Fire"
        elif label[idx][0] == 1 and label[idx][1] == 0:
            title += "Only Smoke"
        elif label[idx][0] == 0 and label[idx][1] == 1:
            title += "Only Fire"
        else:
            title += "Empty"
        plt.title(title)
        
        if (idx == 31):
            break
    plt.tight_layout()
    plt.savefig(config.RUN_FOLDER + 'val_pictures.png')
    plt.close()
    break

# Load Model

In [9]:
# model = cnv_model.MobileNetV2().to(config.DEVICE)
# model = cnv_model.MobileNetV2_SMALL().to(config.DEVICE)

# model = cnv_model.MobileNetV2_MINI_RESNET().to(config.DEVICE)
model = cnv_model.MobileNetV2_MINI().to(config.DEVICE)

### Optimizer and Scheduler

In [10]:
optimizer = optim.Adam(model.parameters(), 
                       lr=config.LEARNING_RATE, 
                       weight_decay=config.WEIGHT_DECAY)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min',
                                                 factor=config.FACTOR, 
                                                 patience=config.PATIENCE, 
                                                 threshold=config.THRES, 
                                                 threshold_mode='abs',
                                                 min_lr=config.MIN_LR)



### Parameters

In [11]:
n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')
logger.info(f'\nTrainable parameters = {n_trainable}')

n_params = parameters_to_vector(model.parameters()).numel()
print(f'Total parameters = {n_params}\n')
logger.info(f'Total parameters = {n_params}\n')


Trainable parameters = 68882
Total parameters = 68882



### Check Model Shape: Random Input

In [12]:
dummy_input = np.random.rand(4, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)
dummy_input = torch.tensor(dummy_input, dtype=torch.float32, device=config.DEVICE)
out_test = model(dummy_input)
print(f'Model shape is {out_test}')
#print(f'BED Model Arquitecture\n{cnv_model}')

Model shape is tensor([[-0.0624,  0.0869],
        [-0.0576,  0.0918],
        [-0.0577,  0.0786],
        [-0.0467,  0.0724]], device='cuda:0', grad_fn=<AddmmBackward0>)


### Torchinfo and Print Model

In [13]:
print(summary(model, input_size=(1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)))

Layer (type:depth-idx)                   Output Shape              Param #
MobileNetV2_MINI                         [1, 2]                    --
├─Sequential: 1-1                        [1, 64, 14, 14]           --
│    └─Sequential: 2-1                   [1, 32, 112, 112]         --
│    │    └─Conv2d: 3-1                  [1, 32, 112, 112]         864
│    │    └─BatchNorm2d: 3-2             [1, 32, 112, 112]         64
│    │    └─ReLU: 3-3                    [1, 32, 112, 112]         --
│    └─InvertedBlock: 2-2                [1, 8, 112, 112]          --
│    │    └─Sequential: 3-4              [1, 8, 112, 112]          624
│    └─InvertedBlock: 2-3                [1, 16, 56, 56]           --
│    │    └─Sequential: 3-5              [1, 16, 56, 56]           624
│    └─InvertedBlock: 2-4                [1, 16, 56, 56]           --
│    │    └─Sequential: 3-6              [1, 16, 56, 56]           1,472
│    └─InvertedBlock: 2-5                [1, 24, 28, 28]           --
│    │   

In [14]:
logger.info("\nTorch Summary")
logger.info(summary(model, input_size=(1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)))

In [15]:
logger.info("\nTorch Model")
logger.info(model)

# Loss Function

In [16]:
if config.LOSS_FN == "BCE":
    print(f'Loss Function: BCE')
    logger.info(f'\nLoss Function: BCE')
    print(f'Smoke Precision Weight: {config.SMOKE_PRECISION_WEIGHT}')
    logger.info(f'Smoke Precision Weight: {config.SMOKE_PRECISION_WEIGHT}')
    loss_fn = loss.BCE_LOSS(device=config.DEVICE, smoke_precision_weight=config.SMOKE_PRECISION_WEIGHT)
else:
    print("Wrong loss function")
    logger.info("Wrong loss function")
    raise SystemExit("Wrong loss function")

Loss Function: BCE
Smoke Precision Weight: 0.8


# Loggers and Plotters for Losses and Metrics

In [17]:
train_losses_logger = utils.LogLosses()
train_metrics_logger = utils.LogMetrics()
lr_logger = utils.LogLR(log_path=config.PLOTS_FOLDER)

val_losses_logger = utils.LogLosses()
val_metrics_logger = utils.LogMetrics()

loss_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Loss')
metrics_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Metric')

# Main Function to Train

In [18]:
def train_loop(model, start_epoch=0, epochs_to_train=config.EPOCHS):

    ''' ==============================================================
                                TRAINING LOOP
    ============================================================== '''
    start = datetime.datetime.now()
    start_time = start.strftime("%H:%M:%S")
    print(f'\n***Start Training: {start_time}\n')
    logger.info(f'\n***Start Training: {start_time}\n')
    
    # Start with infinite validation loss
    best_valid_loss = np.inf
    best_smoke_precision = 0. #torch.tensor([0.])
    smoke_f1_min_save = 0.9 #torch.tensor([0.9])
    best_mean_f1 = 0.

    if start_epoch == 0:
        epochs_plot = []
    else:
        epochs_plot = [i for i in range(start_epoch)]    

    end_epoch = start_epoch + epochs_to_train
        
    for epoch in range(start_epoch, end_epoch):

        print(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        logger.info(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        
        #====================== TRAINING ========================#
        current_lr = train_epoch.get_lr(optimizer=optimizer)
        logger.info(f'Learning Rate = {current_lr}\n')
        lr_logger.log_lr(current_lr)
                
        train_losses, train_metrics = train_epoch.train_fn(
            loader=train_loader, 
            model=model, 
            optimizer=optimizer, 
            loss_fn=loss_fn,
            device=config.DEVICE)
        
        train_losses_logger.update_metrics(train_losses)
        train_metrics_logger.update_metrics(train_metrics)
                
        logger.info(utils.print_metrics_to_logger("TRAIN Stats", train_losses, train_metrics))
        
        #===================== VALIDATING =======================#
        with torch.no_grad():
            val_losses, val_metrics = val_epoch.eval_fn(
                loader=val_loader, 
                model=model,                         
                loss_fn=loss_fn,
                device=config.DEVICE)
            
            scheduler.step(val_losses['Total'])
            
            val_losses_logger.update_metrics(val_losses)
            val_metrics_logger.update_metrics(val_metrics)

            logger.info(utils.print_metrics_to_logger("VAL Stats", val_losses, val_metrics))
            
        epochs_plot.append(epoch)

        loss_plotter.plot_all_metrics(
            train_losses_logger.get_metrics(),
            val_losses_logger.get_metrics(),
            epochs_plot)

        metrics_plotter.plot_all_metrics(
            train_metrics_logger.get_metrics(),
            val_metrics_logger.get_metrics(),
            epochs_plot)

        lr_logger.plot_lr(epochs_plot)
        #======================= SAVING =========================#
        if ( (epoch+1) % 5 ) == 0:
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__5epoch.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            
        if best_valid_loss > val_losses['Total']:
            best_valid_loss = val_losses['Total']
            print(f"\nSaving model with new best validation loss: {best_valid_loss:.4f}")
            logger.info(f"Saving model with new best validation loss: {best_valid_loss:.4f}")
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + 'best_loss'  + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + 'best_loss'  #+ '.onnx'
            # utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)

        # # Save model if precision increases and F1 > 0.9
        # if ( best_smoke_precision < val_metrics['Precision'][0] ) and ( val_metrics['F1'][0] > smoke_f1_min_save ) :
        #     best_smoke_precision = val_metrics['Precision'][0]
        #     print(f"\nSaving model with new best smoke precision: {best_smoke_precision:.4f}")
        #     logger.info(f"Saving model with new best smoke precision: {best_smoke_precision:.4f}")
        #     save_precision_name = f'best_smoke__precision={np.round(best_smoke_precision, decimals=4)}__epoch={epoch}'
        #     save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_precision_name + '.pt'
        #     utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name)  
        #     save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_precision_name #+ '.onnx'
        #     utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)
            
        # Save model if precision > 0.9 and recall > 0.9
        if ( val_metrics['Precision'][0] > 0.9 ) and ( val_metrics['Recall'][0] > 0.9 ) :
            print("\nSaving model with precision > 0.9 and recall > 0.9")
            logger.info("Saving model with precision > 0.9 and recall > 0.9")
            save_pre_name = f'smoke__precision={np.round(val_metrics["Precision"][0], decimals=4)}__' 
            save_rec_name = f'recall={np.round(val_metrics["Recall"][0], decimals=4)}__'
            save_pre_rec_name = save_pre_name + save_rec_name + f'epoch={epoch}'
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_pre_rec_name + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_pre_rec_name #+ '.onnx'
            # utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)

        # Save model if best mean F1 increases
        val_f1_mean = (val_metrics['F1'][0] + val_metrics['F1'][1]) / 2
        if (val_f1_mean > best_mean_f1) :
            best_mean_f1 = val_f1_mean
            print(f'Saving model with best Mean F1: {best_mean_f1:.4f}')
            logger.info(f'Saving model with best Mean F1: {best_mean_f1:.4f}')
            save_f1_name = 'best_mean_F1'
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_f1_name + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_f1_name #+ '.onnx'
            # utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)
        
    logger.info('Saving last model')   
    torch.save(model.state_dict(), config.WEIGHTS_FOLDER + 'last_' + config.MODEL + '_classifier.pt') 
    
    #======================= FINISH =========================#
    end = datetime.datetime.now()
    end_time = end.strftime("%H:%M:%S")
    print(f'\n***Script finished: {end_time}\n')  
    print(f'Time elapsed: {end-start}')
    logger.info(f'\n***Script finished: {end_time}\n')  
    logger.info(f'Time elapsed: {end-start}')
    
    return model

In [19]:
# print(len(val_losses_logger.total))

# Training Loop

In [20]:
if __name__ == "__main__":
    
    print("Starting script\n")
    logger.info("Starting script\n")
    
    model = train_loop(model)

Starting script


***Start Training: 07:21:22


=== EPOCH 0/99 ===
Learning Rate = 0.001



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1836/1836 [05:27<00:00,  5.60it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
43.661      |24.351      |19.310      



Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:22<00:00, 16.66it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
35.152      |21.202      |13.950      
SMOKE -> Precision: 0.8896 - Recall: 0.7014 - Accuracy: 0.8182 - F1: 0.7843
FIRE -> Precision: 0.8361 - Recall: 0.9092 - Accuracy: 0.9128 - F1: 0.8711

Saving model with new best validation loss: 35.1522
Saving model with best Mean F1: 0.8277

=== EPOCH 1/99 ===
Learning Rate = 0.001



...

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
12.792      |7.473       |5.320       
SMOKE -> Precision: 0.9535 - Recall: 0.9403 - Accuracy: 0.9502 - F1: 0.9469
FIRE -> Precision: 0.9329 - Recall: 0.9839 - Accuracy: 0.9718 - F1: 0.9577

Saving model with precision > 0.9 and recall > 0.9

=== EPOCH 99/99 ===
Learning Rate = 9.223372036854789e-06



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1836/1836 [05:24<00:00,  5.66it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
9.611       |5.960       |3.651       



Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:22<00:00, 16.54it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
12.701      |7.440       |5.261       
SMOKE -> Precision: 0.9550 - Recall: 0.9410 - Accuracy: 0.9513 - F1: 0.9479
FIRE -> Precision: 0.9358 - Recall: 0.9834 - Accuracy: 0.9727 - F1: 0.9590

Saving model with precision > 0.9 and recall > 0.9

***Script finished: 17:18:28

Time elapsed: 9:57:05.941435


In [21]:
# if __name__ == "__main__":
    
#     print("Train More script\n")
#     logger.info("Train More\n")
    
#     model = train_loop(model, start_epoch=75, epochs_to_train=15)

# Test with DFire MINI Dataset: Train and Test

In [22]:
# import importlib
# importlib.reload(config)
# importlib.reload(dataloaders)

In [23]:
train_dfire_mini_loader = dataloaders.get_dfire_mini_train_loader()
test_dfire_mini_loader = dataloaders.get_dfire_mini_test_loader()


TRAIN DFire MINI dataset
DFire Removed wrong images: 0
DFire empty images: 20
DFire only smoke images: 45
DFire only fire images: 5
DFire smoke and fire images: 30

Test dataset len: 100

TEST DFire MINI dataset
DFire Removed wrong images: 0
DFire empty images: 6
DFire only smoke images: 13
DFire only fire images: 2
DFire smoke and fire images: 9

Test dataset len: 30


### Load Checkpoint with Best F1 Mean

In [24]:
model_path = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__best_mean_F1.pt'

In [25]:
utils.load_checkpoint(
    model_path, 
    model=model, 
    optimizer=optimizer, 
    scheduler=scheduler, 
    device=config.DEVICE)

Loading Model. Trained during 79 epochs


79

### Whole Test Loader, to check it is the same as training

In [26]:
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=val_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:22<00:00, 16.53it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
12.134      |7.314       |4.820       
SMOKE -> Precision: 0.9592 - Recall: 0.9361 - Accuracy: 0.9511 - F1: 0.9475
FIRE -> Precision: 0.9449 - Recall: 0.9815 - Accuracy: 0.9755 - F1: 0.9629


In [27]:
logger.info('\nTesting with FULL TEST LOADER')  
#logger.info(val_losses)
logger.info(val_metrics)

### Train DFire MINI

In [28]:
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=train_dfire_mini_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)

Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 258.38it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
0.262       |0.145       |0.117       
SMOKE -> Precision: 0.9726 - Recall: 0.9467 - Accuracy: 0.9400 - F1: 0.9595
FIRE -> Precision: 1.0000 - Recall: 0.9429 - Accuracy: 0.9800 - F1: 0.9706


In [29]:
logger.info('\nTesting with DFire MINI TRAIN after LOADING F1 Best Mean CHECKPOINT')  
#logger.info(val_losses)
logger.info(val_metrics)

### Test DFire MINI

In [30]:
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=test_dfire_mini_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)

Validating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 138.52it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
0.162       |0.111       |0.050       
SMOKE -> Precision: 1.0000 - Recall: 0.9091 - Accuracy: 0.9333 - F1: 0.9524
FIRE -> Precision: 1.0000 - Recall: 1.0000 - Accuracy: 1.0000 - F1: 1.0000


In [31]:
logger.info('\nTesting with DFire MINI TEST after LOADING F1 Best Mean CHECKPOINT')  
#logger.info(val_losses)
logger.info(val_metrics)

# Export to ONNX

In [32]:
model.cpu();

In [33]:
torch_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(model, torch_input, config.ONNX_FOLDER + "mbnet_mini.onnx")